In [1]:
# https://www.kaggle.com/brsdincer/heart-attack-prediction-detailed-explanation
import numpy as np
import pandas as  pd
import matplotlib.pyplot as plt
from warnings import filterwarnings
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
import missingno as msno
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import levene
from scipy.stats import shapiro
from scipy.stats.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, ShuffleSplit, GridSearchCV 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
import seaborn as sns 
from sklearn import preprocessing

In [2]:
DATA_PATH = 'C:\\Users\\shota\\Documents\\Data\\Kaggle\\kc_Tabular_Titanick\\'

In [3]:
data = pd.read_csv(DATA_PATH + 'train.csv')

In [4]:
dataT = data.copy()

In [5]:
dataT

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,1,2,"Bell, Adele",female,62.00,0,0,PC 15008,14.86,D17243,C
99996,99996,0,2,"Brown, Herman",male,66.00,0,0,13273,11.15,NaN,S
99997,99997,0,3,"Childress, Charles",male,37.00,0,0,NaN,9.95,NaN,S
99998,99998,0,3,"Caughlin, Thomas",male,51.00,0,1,458654,30.92,NaN,S


In [6]:
len(dataT['Ticket'].unique())

75332

In [7]:
dataT.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1, inplace=True)

In [8]:
dataT

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,1,male,NaN,2,0,27.14,S
1,0,3,male,NaN,0,0,13.35,S
2,0,3,male,0.33,1,2,71.29,S
3,0,3,male,19.00,0,0,13.04,S
4,1,3,male,25.00,0,0,7.76,S
...,...,...,...,...,...,...,...,...
99995,1,2,female,62.00,0,0,14.86,C
99996,0,2,male,66.00,0,0,11.15,S
99997,0,3,male,37.00,0,0,9.95,S
99998,0,3,male,51.00,0,1,30.92,S


In [9]:
dataT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Survived  100000 non-null  int64  
 1   Pclass    100000 non-null  int64  
 2   Sex       100000 non-null  object 
 3   Age       96708 non-null   float64
 4   SibSp     100000 non-null  int64  
 5   Parch     100000 non-null  int64  
 6   Fare      99866 non-null   float64
 7   Embarked  99750 non-null   object 
dtypes: float64(2), int64(4), object(2)
memory usage: 6.1+ MB


In [10]:
dataT = dataT.dropna(how='any').reset_index()

In [11]:
# ダミー変数に変更
for column in ['Sex', 'Embarked']:
    le = preprocessing.LabelEncoder()
    le.fit(dataT[column])
    dataT[column] = le.transform(dataT[column])

In [12]:
dataT

,index,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,2,0,3,1,0.33,1,2,71.29,2
1,3,0,3,1,19.00,0,0,13.04,2
2,4,1,3,1,25.00,0,0,7.76,2
3,5,0,2,1,35.00,0,0,6.71,2
4,6,0,3,1,7.00,0,0,9.77,2
...,...,...,...,...,...,...,...,...,...
96327,99995,1,2,0,62.00,0,0,14.86,0
96328,99996,0,2,1,66.00,0,0,11.15,2
96329,99997,0,3,1,37.00,0,0,9.95,2
96330,99998,0,3,1,51.00,0,1,30.92,2


In [13]:
unsc_data = dataT.copy()

In [14]:
yTrain = unsc_data['Survived']
xTrain = unsc_data.drop(['index','Survived'], axis=1)

In [15]:
# 正規化を行う(feature scaling)
scaler = StandardScaler()
for column in ['Age', 'Fare']:
    scaler.fit(xTrain[column].values.reshape(-1,1))
    xTrain[column] = scaler.transform(xTrain[column].values.reshape(-1,1))

In [16]:
xSTrain = xTrain.copy()

In [17]:
xSTrain

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,-2.076093,1,2,0.393635,2
1,3,1,-1.056683,0,0,-0.444212,2
2,3,1,-0.729074,0,0,-0.520158,2
3,2,1,-0.183059,0,0,-0.535261,2
4,3,1,-1.711901,0,0,-0.491247,2
...,...,...,...,...,...,...,...
96327,2,0,1.291182,0,0,-0.418034,0
96328,2,1,1.509588,0,0,-0.471397,2
96329,3,1,-0.073856,0,0,-0.488658,2
96330,3,1,0.690565,0,1,-0.187033,2


In [18]:
ols = sm.OLS(yTrain, xSTrain).fit()
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               Survived   R-squared (uncentered):                   0.409
Model:                            OLS   Adj. R-squared (uncentered):              0.409
Method:                 Least Squares   F-statistic:                              9526.
Date:                Tue, 06 Apr 2021   Prob (F-statistic):                        0.00
Time:                        14:49:52   Log-Likelihood:                         -70486.
No. Observations:               96332   AIC:                                  1.410e+05
Df Residuals:                   96325   BIC:                                  1.411e+05
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Pclass         0.2245      0.002    144.853      0.000       0.221       0.228
Sex           -0.3243      0.003    -93.889      0.000      -0.331      -0.318
Age            0.0822      0.002     49.893      0.000       0.079       0.085
SibSp          0.0387      0.002     20.202      0.000       0.035       0.042
Parch          0.0441      0.002     25.399      0.000       0.041       0.048
Fare           0.1367      0.002     81.738      0.000       0.133       0.140
Embarked       0.0093      0.002      4.485      0.000       0.005       0.013
==============================================================================
Omnibus:                    26248.003   Durbin-Watson:                   1.945
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4966.030
Skew:                           0.215   Prob(JB):                         0.00
Kurtosis:                       1.974   Cond. No.                         6.36
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
xSTrain_b = xSTrain.copy()

In [20]:
# category型に変更
for column in ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']:
    xSTrain_b[column] = pd.Categorical(xSTrain_b[column])

In [21]:
xSTrain_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96332 entries, 0 to 96331
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Pclass    96332 non-null  category
 1   Sex       96332 non-null  category
 2   Age       96332 non-null  float64 
 3   SibSp     96332 non-null  category
 4   Parch     96332 non-null  category
 5   Fare      96332 non-null  float64 
 6   Embarked  96332 non-null  category
dtypes: category(5), float64(2)
memory usage: 1.9 MB


In [22]:
xSTrain_a = xSTrain_b.copy()

In [28]:
# Classification
lj = LogisticRegression().fit(xSTrain, yTrain)
gnb = GaussianNB().fit(xSTrain, yTrain)
knnc = KNeighborsClassifier().fit(xSTrain, yTrain)
cartc = DecisionTreeClassifier(random_state=42).fit(xSTrain, yTrain)
rfc = RandomForestClassifier(random_state=42, verbose=False).fit(xSTrain, yTrain)
gbmc = GradientBoostingClassifier(verbose=False).fit(xSTrain, yTrain)
xgbc = XGBClassifier().fit(xSTrain, yTrain)
lgbmc = LGBMClassifier().fit(xSTrain, yTrain)
catbc = CatBoostClassifier(verbose=False).fit(xSTrain, yTrain)

C:\Users\shota\anaconda3\envs\DSL_TF2.0\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:04:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [30]:
# Test data, preprocessing
test_data = pd.read_csv(DATA_PATH + 'test.csv')

In [31]:
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,100000,3,"Holliday, Daniel",male,19.0,0,0,24745,63.01,NaN,S
1,100001,3,"Nguyen, Lorraine",female,53.0,0,0,13264,5.81,NaN,S
2,100002,1,"Harris, Heather",female,19.0,0,0,25990,38.91,B15315,C
3,100003,2,"Larsen, Eric",male,25.0,0,0,314011,12.93,NaN,S
4,100004,1,"Cleary, Sarah",female,17.0,0,2,26203,26.89,B22515,C
...,...,...,...,...,...,...,...,...,...,...,...
99995,199995,3,"Cash, Cheryle",female,27.0,0,0,7686,10.12,NaN,Q
99996,199996,1,"Brown, Howard",male,59.0,1,0,13004,68.31,NaN,S
99997,199997,3,"Lightfoot, Cameron",male,47.0,0,0,4383317,10.87,NaN,S
99998,199998,1,"Jacobsen, Margaret",female,49.0,1,2,PC 26988,29.68,B20828,C


In [32]:
test_data.drop(['PassengerId', 'Name', 'Cabin'], axis=1,  inplace=True)

In [34]:
test_data.drop(['Ticket'], axis=1, inplace=True)

In [35]:
test_data

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,19.0,0,0,63.01,S
1,3,female,53.0,0,0,5.81,S
2,1,female,19.0,0,0,38.91,C
3,2,male,25.0,0,0,12.93,S
4,1,female,17.0,0,2,26.89,C
...,...,...,...,...,...,...,...
99995,3,female,27.0,0,0,10.12,Q
99996,1,male,59.0,1,0,68.31,S
99997,3,male,47.0,0,0,10.87,S
99998,1,female,49.0,1,2,29.68,C


In [36]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Pclass    100000 non-null  int64  
 1   Sex       100000 non-null  object 
 2   Age       96513 non-null   float64
 3   SibSp     100000 non-null  int64  
 4   Parch     100000 non-null  int64  
 5   Fare      99867 non-null   float64
 6   Embarked  99723 non-null   object 
dtypes: float64(2), int64(3), object(2)
memory usage: 5.3+ MB


In [42]:
for column in ['Sex', 'Embarked']:
    le = preprocessing.LabelEncoder()
    le.fit(test_data[column])
    test_data[column] = le.transform(test_data[column])

In [45]:
le.classes_    # ダミー化する順序を確認

array(['C', 'Q', 'S', nan], dtype=object)

In [43]:
# AgeのNanを置き換える
test_data = test_data.fillna({'Age':test_data['Age'].median(),
                              'Fare':test_data['Fare'].median(),
                              'Embarked':test_data['Embarked'].median()})

In [44]:
test_data

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,19.0,0,0,63.01,2
1,3,0,53.0,0,0,5.81,2
2,1,0,19.0,0,0,38.91,0
3,2,1,25.0,0,0,12.93,2
4,1,0,17.0,0,2,26.89,0
...,...,...,...,...,...,...,...
99995,3,0,27.0,0,0,10.12,1
99996,1,1,59.0,1,0,68.31,2
99997,3,1,47.0,0,0,10.87,2
99998,1,0,49.0,1,2,29.68,0


In [47]:
scaler = StandardScaler()
for column in ['Age', 'Fare']:
    scaler.fit(test_data[column].values.reshape(-1,1))
    test_data[column] = scaler.transform(test_data[column].values.reshape(-1,1))

In [48]:
test_data

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,-0.827722,0,0,0.271239,2
1,3,0,1.631977,0,0,-0.606451,2
2,1,0,-0.827722,0,0,-0.098557,0
3,2,1,-0.393657,0,0,-0.497200,2
4,1,0,-0.972410,0,2,-0.282994,0
...,...,...,...,...,...,...,...
99995,3,0,-0.248969,0,0,-0.540317,1
99996,1,1,2.066041,1,0,0.352564,2
99997,3,1,1.197912,0,0,-0.528809,2
99998,1,0,1.342601,1,2,-0.240184,0


In [29]:
# COMPARISON
models = [lj, gnb, knnc, cartc, rfc, gbmc, xgbc, lgbmc, catbc]

In [49]:
for model in models:
    name = model.__class__.__name__
    predict = model.predict(xSTrain)
    print(name)
    print(accuracy_score(yTrain, predict))
    print('--------'*20)

LogisticRegression
0.7666403687248267
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
GaussianNB
0.737158991819956
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
KNeighborsClassifier
0.8133226757463771
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
DecisionTreeClassifier
0.9930864095004776
--------------------------------------------------------------------------------------------------------------------------------

In [54]:
# create submission
conclusion_data = pd.read_csv(DATA_PATH + 'sample_submission.csv')
conclusion_data['Survived'] = xgbc.predict(test_data)
conclusion_data.to_csv('kc_Tabular_Titanic.csv', index=False)

In [58]:
import tensorflow as tf

In [59]:
# 訓練用と検証用に分ける
xTR, xV, yTR, yV = train_test_split(xSTrain, yTrain, test_size=0.2, random_state=42) 

In [76]:
# Tensorflowで挑戦
input_size=len(xSTrain.columns)
output_size=2
hidden_layer_size=64

modelT = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

modelT.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size=100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=4)

modelT.fit(xTR,
          yTR,
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(xV, yV),
          verbose=2
          )

Epoch 1/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

771/771 - 3s - loss: 0.5042 - accuracy: 0.7623 - val_loss: 0.4958 - val_accuracy: 0.7649
Epoch 2/100
771/771 - 2s - loss: 0.4950 - accuracy: 0.7667 - val_loss: 0.5021 - val_accuracy: 0.7635
Epoch 3/100
771/771 - 2s - loss: 0.4936 - accuracy: 0.7685 - val_loss: 0.4927 - val_accuracy: 0.7668
Epoch 4/100
771/771 - 2s - loss: 0.4924 - accuracy: 0.7683 - val_loss: 0.4925 - val_accuracy: 0.7674
Epoch 5/100
771/771 - 2s - loss: 0.4914 - accuracy: 0.7703 - val_loss: 0.4938 - val_accuracy: 0.7666
Epo

In [77]:
# create submission
conclusion_data = pd.read_csv(DATA_PATH + 'sample_submission.csv')
conclusion_data['Survived'] = modelT.predict(test_data).round().astype(int)[:,1]
conclusion_data.to_csv('kc_Tabular_Titanic_Te.csv', index=False)

In [71]:
modelT.predict(test_data).round().astype(int)[:,1]

array([0, 1, 1, ..., 0, 1, 1])

In [73]:
modelT.predict(test_data).round().astype(int)
# 左が0,右が1

array([[1, 0],
       [0, 1],
       [0, 1],
       ...,
       [1, 0],
       [0, 1],
       [0, 1]])